In [5]:
%%time
import segmind.instance as instance
import requests
import traceback
from datetime import datetime

try:
    import numpy as np
    import scipy.linalg as la
    from numpy import angle, pi
    from qutip import Qobj, propagator

    def floquet_modes_mod(H, T, args=None, parallel=False, sort=False, U=None):
        """
        Calculate the initial Floquet modes Phi_alpha(0) for a driven system with
        period T.

        Returns a list of :class:`qutip.qobj` instances representing the Floquet
        modes and a list of corresponding quasienergies, sorted by increasing
        quasienergy in the interval [-pi/T, pi/T]. The optional parameter `sort`
        decides if the output is to be sorted in increasing quasienergies or not.

        Parameters
        ----------

        H : :class:`qutip.qobj`
            system Hamiltonian, time-dependent with period `T`

        args : dictionary
            dictionary with variables required to evaluate H

        T : float
            The period of the time-dependence of the hamiltonian. The default value
            'None' indicates that the 'tlist' spans a single period of the driving.

        U : :class:`qutip.qobj`
            The propagator for the time-dependent Hamiltonian with period `T`.
            If U is `None` (default), it will be calculated from the Hamiltonian
            `H` using :func:`qutip.propagator.propagator`.

        Returns
        -------

        output : list of kets, list of quasi energies

            Two lists: the Floquet modes as kets and the quasi energies.

        """
        if 'opts' in args:
            options = args['opts']
        else:
            options = Options()
            options.rhs_reuse = True
            rhs_clear() 

        if U is None:
            # get the unitary propagator
            U = propagator(H, T, [], args, parallel=parallel, progressbar=True, options=options)

        # find the eigenstates for the propagator
        evals, evecs = la.eig(U.full())

        eargs = angle(evals)

        # make sure that the phase is in the interval [-pi, pi], so that
        # the quasi energy is in the interval [-pi/T, pi/T] where T is the
        # period of the driving.  eargs += (eargs <= -2*pi) * (2*pi) +
        # (eargs > 0) * (-2*pi)
        eargs += (eargs <= -pi) * (2 * pi) + (eargs > pi) * (-2 * pi)
        e_quasi = -eargs / T

        # sort by the quasi energy
        if sort:
            order = np.argsort(-e_quasi)
        else:
            order = list(range(len(evals)))

        # prepare a list of kets for the floquet states
        new_dims = [U.dims[0], [1] * len(U.dims[0])]
        new_shape = [U.shape[0], 1]
        kets_order = [Qobj(np.matrix(evecs[:, o]).T,
                           dims=new_dims, shape=new_shape) for o in order]

        return kets_order, e_quasi[order]
    
    from qutip import mesolve, basis, jmat

    def drive(t, args):
        w = args['omega']
        h = args['h']
        h0 = args['h0']
        return h * np.cos(w*t) + h0

    def get_hamiltonians(N):
        sx,sy,sz = jmat(N,"x"),jmat(N,"y"),jmat(N,"z")
        kn =  2.0/(N-1)                                      # kacNorm
        H0 = kn * sz **2 
        H1 = 2 * sx
        return H0,H1

    def floq_evolv(args):
        N = args['N']
        T = 2 * np.pi/args['omega']
        opts = args['opts']
        H0, H1 = get_hamiltonians(N)
        H = [H0,[H1,drive]]
        f_states, _ = floquet_modes_mod(H, T, args=args)
        return f_states

  
        # Lipkin Model Cross-Section
    from scipy.integrate import odeint, odepack
    import matplotlib.pyplot as plt
    from multiprocessing import Pool
    from scipy.special import j0, jn_zeros
    import warnings

    warnings.filterwarnings("ignore")
    from qutip import Options
    from qutip.wigner import qfunc

    # Maximum number of strobes
    nmax = 1000
    # Number of Processors
    nprocs = 11

    def lmg(z, t, h0, h, w):
        q, p = z
        drive = h0 + h * np.cos(w * t)
        sterm = np.sqrt(1.0 - 4.0 * q * q)
        qdot = drive * sterm * np.sin(p)
        pdot = 4.0 * q * (1.0 - (drive * np.cos(p)/sterm) )
        return [qdot, pdot]

    def poincare_sec(x0, p0, nmax, args):
        h0, h, omega = args
        T = 2.0 * np.pi/omega
        t = T * np.arange(nmax)
        init = [x0,p0]
        points = odeint(lmg, init, t, args=args)
        points[:,1] = points[:,1] % (2.0 * np.pi)
        return points

    plt.rcParams.update({
        "figure.figsize": (20, 20),
        "text.usetex": True,
        "font.family": "sans-serif",
        "font.size": 20,
        "font.sans-serif": ["Computer Modern"]})
    
    
    print("Definitions complete!")


    # Initial Conditions for classical poincare
    n_ic = 15
    init_x = np.linspace(-0.45, 0.45, n_ic)
    near_zero = np.linspace(-0.01, 0.01, 10)
    init_x = np.sort(np.append(init_x, near_zero))
    init_p = np.linspace(0.0, 2.0 * np.pi, n_ic)
    init_p = np.sort(np.append(init_p, [0.1, np.pi + 0.2]))
    xv, pv = np.meshgrid(init_x, init_p, indexing='ij')

    # Initial Conditions for quantum husimi
    # Choose a size
    N = 150
    #Qutip Solver Options
    opts = Options(nsteps=1e5, num_cpus=13, openmp_threads=13)
    n_Q = 100
    Q_qvals = np.linspace(-0.5, 0.5, n_Q)
    Q_pvals = np.linspace(0.0, 2.0 * np.pi, n_Q)
    Q_pvals = np.sort(np.append(Q_pvals, [0.1, np.pi + 0.2]))
    
    # Drive parameters
    h0 = 0.0
    omegas = [2.5, 90.0]
    eta = jn_zeros(0,1)
    eta = eta[0]
    
    n_omegas = len(omegas)
    fig, axs = plt.subplots(2, n_omegas, figsize=(15, 10))

    for i, w in enumerate(omegas):
        h_frz = eta * w/4
        print(f"Poincare section dynamics for h = {h_frz} w = {w} with {nprocs} processes ...")   
        start = time.time()
        with Pool(processes = nprocs) as p:
            poincare_data = p.starmap(poincare_sec,[(x0, p0, nmax, (h0, h_frz, w)) for x0, p0 in zip(xv.flatten(), pv.flatten())])
        
        poincare_data = np.vstack(poincare_data)
        qvals = poincare_data[:,0]
        pvals = poincare_data[:,1]
        elapsed =  (time.time() - start)
        print(f"Done in {elapsed} secs")
        
        axs[0,i].set_title(r'$N={:d}$ (Husimi), $\omega = {:2.1f}, J_0(4 h/\omega) = {:1.3f}$'.format(N,w,j0(4*h_frz/w)))
        axs[0,i].scatter(qvals, pvals , color='blue', marker='.', s=.1)  
        
        #Now for the quantum case
        print(f"Husimi plots for h = {h_frz} w = {w} with {nprocs} processes ...")  
        start = time.time()
        params = {'h0':0, 'h':h_frz, 'omega':w, 'N':N, 'opts':opts} 
        H0, H1 = get_hamiltonians(N)
        H = [H0,[H1,drive]]
        T = 2 * np.pi/w
        evecs_f, _ = floquet_modes_mod(H, T, args=params)
        Q_fstate = 0
        for fstate in evecs_f:
            Q_fstate += qfunc(fstate, Q_qvals, Q_pvals)
        
        elapsed =  (time.time() - start)
        print(f"Done in {elapsed} secs")
        axs[1,i].contourf(Q_qvals, Q_pvals, Q_fstate/(2*N+1), 100)
        axs[1,i].set_xlabel(r'$q$')
    
    
    #Formatting
    x_tick = np.array([-0.5, 0.0, 0.5])
    unit   = 1.0
    p_tick = np.arange(0.0, 2.0 + unit, unit)
    p_label = [r"$0$", r"$\pi$", r"$2\pi$"]
    
    for ax in axs.flatten():
        ax.set_xticks(x_tick)
        ax.set_xlim(-0.52, 0.52)
        ax.set_yticks(p_tick*np.pi, p_label)
        ax.set_ylabel(r"$p$")
        ax.tick_params(direction='in', length=8, width=3)
    
    #plt.savefig("lmg_poincare01.svg")
    plt.show()
    
    #print("Exact simulation complete! Saving Data ...")
    #import h5py
    #dt = f'{datetime.now():%m_%d_%Y_%H_%M_%S}'
    #fname = "iprdata_" + dt + ".hdf5"
    #with h5py.File(fname, 'w') as hf:
    #    ipr_dset =  hf.create_dataset("iprmat",np.shape(iprmat),  data=iprmat)
    #    ampl_dset = hf.create_dataset("amplitudes", np.shape(hs), data=hs)
    #    ampl_dset.attrs['omega'] = omega
    #    ampl_dset.attrs['N'] = N
    #################################################
    bot_message = "Code executed with no exceptions.\n"
except Exception:
    # This bit exists in case your code conks out with an exception
    # That way, the exception will no longer terminate your execution
    # without stopping the segmind instance
    print(traceback.format_exc())
    bot_message = f"Code exited with {Exception}. Check stderr for details.\n"

#####################################################
# You can customize the bot message as per your needs
#####################################################
whoami = "Analabha Roy"
# datetime object containing current date and time
now = datetime.now()
bot_message = 'Program run by ' + whoami + ' has been completed. Date/Time of termination:' + str(now) + bot_message
#####################################################

# The SegmindStatusBot is sent an instruction to broadcast the message in 'bot_message'
bot_token = 'PUT BOT TOKEN'
bot_chatID = 'PUT CHAT ID'
send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
#response = requests.get(send_text)

from ipylab import JupyterFrontEnd
app = JupyterFrontEnd()

app.commands.execute('docmanager:save')

# The running instance is now automatically stopped so that it doesn't eat any more credits than it has to
#instance.stop()
print("Done!")

Definitions complete!
Running for N =  10 with 4 processors


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 103/103 [00:00<00:00, 32346.01it/s]



Done. Now calculating PR ...
Exact simulation complete! Saving Data ...
Done!
CPU times: user 29.3 ms, sys: 14.2 ms, total: 43.5 ms
Wall time: 12.8 s
